In [1]:
import time
from matplotlib import pyplot as plt
import numpy as np
import mxnet as mx
from mxnet import autograd, gluon
import gluoncv as gcv
from gluoncv.utils import download, viz
import os
from mxnet.gluon.data import DataLoader
from gluoncv.data.transforms import presets
from gluoncv.data import RecordFileDetection

In [6]:
train_data = gcv.data.RecordFileDetection('./Deep-Waldo/data/train.rec')
valid_data = gcv.data.RecordFileDetection('./Deep-Waldo/data/valid.rec')

classes = ['Waldo'] 

In [7]:
def get_dataloader(net, train_dataset, data_shape, batch_size, num_workers):
    from gluoncv.data.batchify import Tuple, Stack, Pad

    width, height = data_shape, data_shape

    with autograd.train_mode():
        batchify_fn = Tuple(*([Stack() for _ in range(6)] + [Pad(axis=0, pad_val=-1) for _ in range(1)]))
        train_transform = gcv.data.transforms.presets.rcnn.FasterRCNNDefaultTrainTransform(net=net)
        train_loader = DataLoader(train_dataset.transform(train_transform), batch_size, shuffle=True,
                          batchify_fn=batchify_fn, last_batch='rollover', num_workers=num_workers)
    return train_loader
train = get_dataloader(net, train_data, 128, 32, 0) # look into modifying this because what is our data sahpe?


In [8]:
net = gcv.model_zoo.get_model('faster_rcnn_resnet50_v1b_voc', pretrained=True)

net.reset_class(classes)

In [9]:
try:
    ctx = [mx.gpu()]
    print('GPU Loaded')
except:
    ctx = [mx.cpu()]

GPU Loaded


In [ ]:
d

In [10]:
net.collect_params().reset_ctx(ctx)

trainer = gluon.Trainer(
    net.collect_params(), 'adam',
    {'learning_rate': 0.001, 'wd': 0.0005})

mbox_loss = mx.gluon.loss.SigmoidBinaryCrossEntropyLoss(from_sigmoid=False)
ce_metric = mx.metric.Loss('CrossEntropy')
smoothl1_metric = mx.metric.Loss('SmoothL1')

num_epochs = 10

for epoch in range(num_epochs):
  
    ce_metric.reset() 
    smoothl1_metric.reset()
    tic = time.time()
    btic = time.time()
    net.hybridize(static_alloc=True, static_shape=True) # look this up if you wan tot know more https://mxnet.incubator.apache.org/api/python/gluon/gluon.html?highlight=hybridize#mxnet.gluon.Block.hybridize
    
    for i, batch in enumerate(train_data):

        batch_size = batch[0].shape[0]
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0)
        cls_targets = gluon.utils.split_and_load(batch[1][:,-1], ctx_list=ctx, batch_axis=0)
        box_targets = gluon.utils.split_and_load(batch[1][:,0:4], ctx_list=ctx, batch_axis=0)
        
        with autograd.record():
            cls_preds = []
            box_preds = []
            for x in data:
                print(x)
                cls_pred, box_pred, _ = net(x)
                cls_preds.append(cls_pred)
                box_preds.append(box_pred)
            sum_loss, cls_loss, box_loss = mbox_loss(
                cls_preds, box_preds, cls_targets, box_targets)
            autograd.backward(sum_loss)

            trainer.step(1)
        ce_metric.update(0, [l * batch_size for l in cls_loss])
        smoothl1_metric.update(0, [l * batch_size for l in box_loss])
        name1, loss1 = ce_metric.get()
        name2, loss2 = smoothl1_metric.get()
        if i % 20 == 0:
            print('[Epoch {}][Batch {}], Speed: {:.3f} samples/sec, {}={:.3f}, {}={:.3f}'.format(
                epoch, i, batch_size/(time.time()-btic), name1, loss1, name2, loss2))
        btic = time.time()
        
    if not os.path.isdir('./model'):
        os.mkdir('./model')
      
    net.save_parameters('./model/waldo.params')
    


[[[239 237  28]
  [234 225  10]
  [255 237  14]
  ..., 
  [112  81  99]
  [ 97  64  95]
  [ 18   0  23]]

 [[242 238  68]
  [224 213  33]
  [242 216  17]
  ..., 
  [ 17   0  20]
  [ 17   0  25]
  [ 23   5  31]]

 [[ 89  83   0]
  [235 224 108]
  [255 247  94]
  ..., 
  [ 10  17  45]
  [  2   9  25]
  [ 11  20  27]]

 ..., 
 [[253 254 255]
  [253 254 255]
  [253 255 252]
  ..., 
  [252 255 255]
  [254 255 255]
  [254 255 255]]

 [[254 254 254]
  [254 254 254]
  [254 254 254]
  ..., 
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[254 254 254]
  [254 254 254]
  [254 254 254]
  ..., 
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
<NDArray 1251x2048x3 @gpu(0)>


AssertionError: Unknown input to HybridBlock: fasterrcnn1_rcnntargetsampler0_slice_axis2_data

In [11]:
from matplotlib import pyplot as plt
import gluoncv
from gluoncv import model_zoo, data, utils

In [ ]:
net = model_zoo.get_model('faster_rcnn_resnet50_v1b_coco', pretrained=True)

In [12]:
train_data = gcv.data.RecordFileDetection('./Deep-Waldo/data/train.rec')
valid_data = gcv.data.RecordFileDetection('./Deep-Waldo/data/valid.rec')

classes = ['Waldo'] 

In [ ]:
image, labels = train_data[idx]

In [ ]:
x, orig_img = data.transforms.presets.rcnn.load_test()

In [ ]:
net.reset_classes(classes=['Waldo'], reuse_wegihts)